In [1]:
from brute_force import *
from utils import *
import pandas as pd
from embedding_utils import *

In [2]:
qpu_chimera = DWaveSampler(solver={"name": "DW_2000Q_6"}).to_networkx_graph()
qpu_pegasus = get_qpu()

In [3]:
num_variables, num_clauses, variables, clauses = load_cnf(f"./sgen_cnfs/var100.cnf")
count_long_clauses = 0
for clause in clauses:
    if len(clause) > 2:
        count_long_clauses+=1
count_long_clauses

40

In [5]:
variances_pegasus = []
qubits_used_pegasus = []
max_chain_length_pegasus = []
variances_chimera = []
qubits_used_chimera = []
max_chain_length_chimera = []
nvars = []
nclauses = []
seeds_chimera = []
seeds_pegasus = []
bqm_sizes = []

def add_embedding_statistics(embedding, variances, qubits_used, max_chain_lengths):
    max_chain_length, qubits, current_variance = get_embedding_statistics(embedding)
    variances.append(current_variance)
    qubits_used.append(qubits)
    max_chain_lengths.append(max_chain_length)

for i in range(0, count_long_clauses+1, 5):
    num_variables, num_clauses, variables, clauses = load_cnf(f"./sgen_cnfs/var100.cnf")
    nvars.append(num_variables)
    nclauses.append(num_clauses)
    bqm = create_all_variables(variables)
    clauses_to_flip = i
    for clause in clauses:
        if len(clause) > 2 and clauses_to_flip > 0:
            bqm.update(only_one_true(clause))
            clauses_to_flip -= 1
        else:
            clause_to_bqm(bqm, variables, clause, 1)
    bqm_sizes.append(len(bqm.variables))
    best_embedding_pegasus, best_pegasus_seed = find_best_embedding(bqm, qpu_pegasus, isnetworkx=False)
    #best_embedding_chimera, best_chimera_seed = find_best_embedding(bqm, qpu_chimera, isnetworkx=True)
    #seeds_chimera.append(best_chimera_seed)
    seeds_pegasus.append(best_pegasus_seed)
    #add_embedding_statistics(best_embedding_chimera, variances_chimera, qubits_used_chimera, max_chain_length_chimera)
    add_embedding_statistics(best_embedding_pegasus, variances_pegasus, qubits_used_pegasus, max_chain_length_pegasus)

df = pd.DataFrame({
    'vars': nvars,
    'clauses': nclauses,
    'bqm_vars': bqm_sizes,
    'max_chain_length_p': max_chain_length_pegasus,
    #'max_chain_length_c': max_chain_length_pegasus,
    'variance_p': variances_pegasus,
    #'variance_c':variances_chimera,
    'qubits_p': qubits_used_pegasus,
    #'qubits_c': qubits_used_chimera,
    'seed_p': seeds_pegasus,
    #'seed_c': seeds_chimera
})

df
    

best embedding random_seed 8
best embedding max_chain_length 9
best embedding qubits used 781
best embedding variance: 3.3353881278538813
best embedding random_seed 30
best embedding max_chain_length 8
best embedding qubits used 674
best embedding variance: 3.1471544715447153
best embedding random_seed 84
best embedding max_chain_length 8
best embedding qubits used 685
best embedding variance: 3.763993316624896
best embedding random_seed 53
best embedding max_chain_length 8
best embedding qubits used 684
best embedding variance: 2.888144499178982
best embedding random_seed 83
best embedding max_chain_length 9
best embedding qubits used 664
best embedding variance: 3.4742138364779875
best embedding random_seed 27
best embedding max_chain_length 9
best embedding qubits used 649
best embedding variance: 3.306704980842912
best embedding random_seed 8
best embedding max_chain_length 8
best embedding qubits used 588
best embedding variance: 3.1661299940369707
best embedding random_seed 69
be

,vars,clauses,bqm_vars,max_chain_length_p,variance_p,qubits_p,seed_p
0,100,240,220,9,3.34,781,8
1,100,240,205,8,3.15,674,30
2,100,240,190,8,3.76,685,84
3,100,240,175,8,2.89,684,53
4,100,240,160,9,3.47,664,83
5,100,240,145,9,3.31,649,27
6,100,240,130,8,3.17,588,8
7,100,240,115,9,2.26,601,69
8,100,240,100,9,2.11,557,9
